In [1]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
import os
import warnings
from google.colab import files

warnings.filterwarnings('ignore', category=UserWarning)

def proses_pembagian(df):
    """Fungsi utama untuk melakukan seluruh proses pembagian."""
    print("Memulai pra-pemrosesan data...")
    df_aktif = df[df['STATUS AKTIF'] == 'Terdaftar'].copy()
    if df_aktif.empty:
        print("❌ ERROR: Tidak ada mahasiswa dengan 'STATUS AKTIF' ditemukan di dalam file.")
        return None

    df_aktif['stratify_col'] = df_aktif['PROGRAM STUDI'] + '_' + df_aktif['JENIS KELAMIN']

    N_CLUSTERS = 25
    N_GROUPS_PER_CLUSTER = 4

    print(f"Total Mahasiswa Aktif ditemukan: {len(df_aktif)}")
    print("Rekapitulasi Mahasiswa Aktif:")
    print(pd.crosstab(df_aktif['PROGRAM STUDI'], df_aktif['JENIS KELAMIN']))
    print("-" * 30)

    print("\nMemulai pembagian ke dalam Cluster...")
    skf_cluster = StratifiedKFold(n_splits=N_CLUSTERS, shuffle=True, random_state=42)
    df_aktif['Cluster'] = 0
    for i, (_, test_index) in enumerate(skf_cluster.split(df_aktif, df_aktif['stratify_col'])):
        df_aktif.loc[df_aktif.index[test_index], 'Cluster'] = i + 1
    print("✅ Pembagian Cluster Selesai.")
    print("-" * 30)

    print("\nMemulai pembagian ke dalam Kelompok di setiap cluster (dengan penomoran berurutan)...")
    df_aktif['Kelompok'] = 0
    global_group_counter = 0
    for cluster_id in range(1, N_CLUSTERS + 1):
        cluster_data = df_aktif[df_aktif['Cluster'] == cluster_id]
        skf_group = StratifiedKFold(n_splits=N_GROUPS_PER_CLUSTER, shuffle=True, random_state=42)
        try:
            for i, (_, test_index) in enumerate(skf_group.split(cluster_data, cluster_data['stratify_col'])):
                global_group_counter += 1
                df_aktif.loc[cluster_data.index[test_index], 'Kelompok'] = global_group_counter
        except ValueError:
            print(f"⚠️ Peringatan: Stratifikasi gagal di Cluster {cluster_id}, menggunakan metode acak.")
            indices = np.array_split(np.random.permutation(cluster_data.index), N_GROUPS_PER_CLUSTER)
            for i, group_indices in enumerate(indices):
                global_group_counter += 1
                df_aktif.loc[group_indices, 'Kelompok'] = global_group_counter

    print("✅ Pembagian Kelompok Selesai.")
    print("-" * 30)

    hasil_final = df_aktif[['Cluster', 'Kelompok', 'NIM', 'NAMA', 'JENIS KELAMIN', 'PROGRAM STUDI', 'NO HP']].copy()
    hasil_final.sort_values(by=['Cluster', 'Kelompok', 'PROGRAM STUDI'], inplace=True)

    return hasil_final

def main():
    """Fungsi utama untuk menjalankan keseluruhan skrip."""
    print("Unggah file data Mahasiswa Baru (CSV atau Excel)...")
    uploaded = files.upload()

    if not uploaded:
        print("Tidak ada file yang diunggah. Program berhenti.")
        return

    file_path = list(uploaded.keys())[0]
    print(f"File yang diunggah: {file_path}")

    try:
        if file_path.endswith('.csv'):
            df = pd.read_csv(file_path)
        elif file_path.endswith('.xlsx'):
            df = pd.read_excel(file_path)
        else:
            print("❌ ERROR: Format file tidak didukung. Harap gunakan .csv atau .xlsx")
            os.remove(file_path)
            return
        print("Data berhasil dimuat.\n")
    except Exception as e:
        print(f"❌ ERROR: Gagal memuat file. Pastikan file tidak rusak atau formatnya benar. Detail: {e}")
        os.remove(file_path)
        return

    hasil_final = proses_pembagian(df)

    if hasil_final is not None:
        try:
            output_filename = 'pembagian_kelompok_final.csv'
            hasil_final.to_csv(output_filename, index=False)
            print(f"\n🎉 Selamat! Seluruh proses selesai.")
            print(f"Hasil pembagian telah disimpan di file: '{os.path.abspath(output_filename)}'")
            files.download(output_filename)
        except Exception as e:
            print(f"❌ ERROR: Gagal menyimpan file hasil. Detail: {e}")
        finally:
            if os.path.exists(file_path):
                os.remove(file_path)


if __name__ == "__main__":
    main()

Unggah file data Mahasiswa Baru (CSV atau Excel)...


KeyboardInterrupt: 